In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.metrics import  silhouette_score
from sklearn import datasets
from scipy.cluster.hierarchy import dendrogram, linkage
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import KElbowVisualizer

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 50
pd.options.display.max_rows = 200

In [ ]:
data = pd.read_csv('../data/data_clean.csv', index_col=[0])

In [ ]:
df = data.copy()
df.dropna(inplace=True)

In [ ]:
# true/false cols to 1/0
df['variant'] = df['variant'].apply(int)
df['first_appear_event'] = df['first_appear_event'].apply(int)
df['death_event'] = df['death_event'].apply(int)
df['origin_event'] = df['origin_event'].apply(int)

In [ ]:
# create months_ago column
df['months_ago'] = df['date'].apply(lambda x: (2022-int(x[:4]))*12+(1-int(x[5:7])))

In [ ]:
# create volume_type
df['volume_type'] = df['volume'][df['volume']>'9']
df['volume_type'] = df['volume_type'].fillna('ongoing')

In [ ]:
# remove volume types
df = df[~df['volume_type'].isin(['TPB/SC', 'HC', 'GN'])]

In [ ]:
# set '0' to string 'volume'
df['volume'][df['volume']>'9'] = '0'
df['volume'] = df['volume'].apply(int)

In [ ]:
# drop cover price >9
df = df[df['cover_price']<9]

In [ ]:
# drop years
df['year'] = df['date'].str[:4].apply(int)
df = df[df['year']>1935]
df = df[df['year']!=2022]

In [ ]:
# 0 cover price to year mean
d = dict(df.groupby('year').mean()['cover_price'].sort_index())
df['cover_price'][df['cover_price']==0] = df['year'].map(d)

In [ ]:
# create current_value/cover_price
df['current_value'][df['current_value']==0] = df['cover_price']
df['value_over_price'] = df['current_value']/df['cover_price']

In [ ]:
# create pub_issues_total/pub_titles_total
df['pub_issues_per_title'] = df['pub_issues_total']/df['pub_titles_total']

In [ ]:
# one hit encoding of volume_type, special
df = df.join(pd.get_dummies(df['volume_type'], prefix='volume_type'))
df = df.join(pd.get_dummies(df['special'], prefix='special'))

In [ ]:
# drop unwanted columns
df = df.sample(10000)
df_keep = df.drop(columns=['year', 'cover_price', 'current_value', 'pub_titles_total', 'pub_name', 'title', 'title_id', 'issues_total', 'volume_type', 'special', 'date'])

In [ ]:
# create X scaled
ss = StandardScaler()
ss.fit(df_keep)
X = ss.transform(df_keep)

In [ ]:
model_kmeans = KMeans(n_clusters=4, random_state=526).fit(X)

In [ ]:
silhouette_score(X, model_kmeans.labels_)

In [ ]:
visualizer = SilhouetteVisualizer(model_kmeans, colors='yellowbrick')
visualizer.fit(X)
visualizer.poof()

In [ ]:
visualizer = KElbowVisualizer(model_kmeans, k=(2,10))
visualizer.fit(X)
visualizer.show()

In [ ]:
# put labels on dataset
y = model_kmeans.labels_
df['label'] = y

In [ ]:
df['label'].value_counts().sort_index()

In [ ]:
df.groupby('label').mean()

In [ ]:
'''

'0' = high issue_nr , low variant , no volume , low current_value , low searched, low owned , no events , old , magazine
'1' = low issue_nr , no volume , low current_value , limited_series
'2' = higher volume , medium current_value , low searched , low owned , medium old , volume type special+ongoing  
'3' = high variant , high cover price , low owned , newer , SPECIAL
'4' = high current_value , high searched , high owned , high events , high pub_titles_total+issues


'0' = old unpopular magazines
'1' = unpopular limited
'2' = medium 
'3' = newer variant of / special
'4' = popular, high value, events


'''